In [5]:
import sys
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn import datasets
from xgboost import XGBRegressor,XGBClassifier
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
sys.path.append('C:\\DATA_SCIENCE')
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold, KFold, cross_val_score, GridSearchCV, cross_validate 
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, recall_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, RobustScaler, Normalizer, PolynomialFeatures, MinMaxScaler
from LIBRARY.Libreria_Folders_DVG import *
from LIBRARY.Libreria_Pandas_DVG import *
from LIBRARY.Libreria_ML_DVG import *
from LIBRARY.Libreria_Maths_DVG import *
from LIBRARY.Libreria_Graphs_DVG import *

In [6]:
import pymysql
from ml_lib.to_ML import *
from eda_lib.eda import *
from sql_lib import sql_connector as sql

In [7]:
conn = sql.connector()

In [8]:
cursor = conn.cursor()

In [9]:
cursor.execute("SHOW TABLES") 
cursor.fetchall()

[{'Tables_in_australia_fires_2': 'fire_archive_M6_96619'},
 {'Tables_in_australia_fires_2': 'fire_archive_V1_96617'},
 {'Tables_in_australia_fires_2': 'fire_nrt_M6_96619'},
 {'Tables_in_australia_fires_2': 'fire_nrt_V1_96617'},
 {'Tables_in_australia_fires_2': 'student_findings'}]

In [6]:
query = """
SELECT *
FROM fire_archive_M6_96619
"""
am6 = pd.read_sql_query(query, conn)

In [7]:
query2 = """
SELECT *
FROM fire_archive_V1_96617
"""
av1 = pd.read_sql_query(query2, conn)

In [8]:
query3 = """
SELECT *
FROM fire_nrt_M6_96619
"""
nrt_m6 = pd.read_sql_query(query3, conn)

In [9]:
query4 = """
SELECT *
FROM fire_nrt_V1_96617
"""
nrt_v1 = pd.read_sql_query(query4, conn)

In [10]:
print(am6.isnull().sum(), print(av1.isnull().sum(), nrt_m6.isnull().sum(), nrt_v1.isnull().sum()))  #no Nans

index         0
latitude      0
longitude     0
bright_ti4    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_ti5    0
frp           0
fire_type     0
dtype: int64 index         0
latitude      0
longitude     0
brightness    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_t31    0
frp           0
daynight      0
dtype: int64 index         0
latitude      0
longitude     0
bright_ti4    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_ti5    0
frp           0
daynight      0
dtype: int64
index         0
latitude      0
longitude     0
brightness    0
scan          0
track         0
acq_date      0
acq_time      0
satellite     0
instrument    0
confidence    0
version       0
bright_t31    0
frp           0
daynight      0
f

In [11]:
am6.shape, av1.shape, nrt_m6.shape, nrt_v1.shape

((36011, 16), (184778, 15), (183593, 15), (956257, 15))

In [12]:
am6.columns.difference(nrt_m6.columns), av1.columns.difference(nrt_v1.columns)

(Index(['fire_type'], dtype='object'), Index(['fire_type'], dtype='object'))

In [13]:
am6.columns.difference(av1.columns)

Index(['bright_t31', 'brightness', 'daynight'], dtype='object')

In [14]:
le = LabelEncoder()
global le                                           #asignamos el mismo lable encoder para todos los dataframe para homogeneizar
def encoder2(df):
    cat_cols = (df.dtypes =="object")
    object_cols = list(cat_cols[cat_cols].index)
    print("Categorical variables:")
    print(object_cols)
    for col in object_cols:
        df[col] = le.fit_transform(df[col])

In [15]:
encoder2(am6), encoder2(av1), encoder2(nrt_m6), encoder2(nrt_v1)

Categorical variables:
['latitude', 'acq_date', 'satellite', 'instrument', 'daynight']
Categorical variables:
['latitude', 'acq_date', 'satellite', 'instrument', 'confidence']
Categorical variables:
['latitude', 'acq_date', 'satellite', 'instrument', 'version', 'daynight']
Categorical variables:
['latitude', 'acq_date', 'satellite', 'instrument', 'confidence', 'version', 'daynight']


(None, None, None, None)

In [16]:
target_corr(av1, 'fire_type')

latitude      0.037375
longitude     0.001074
bright_ti4   -0.032832
scan         -0.008146
track        -0.011568
acq_date      0.012548
acq_time      0.028850
satellite          NaN
instrument         NaN
confidence    0.021982
version            NaN
bright_ti5   -0.044027
frp          -0.018583
fire_type     1.000000
Name: fire_type, dtype: float64


In [17]:
target_corr(am6, 'fire_type')

latitude      0.063429
longitude     0.008290
brightness   -0.052514
scan         -0.023828
track        -0.024296
acq_date     -0.004873
acq_time      0.001431
satellite     0.021261
instrument         NaN
confidence   -0.039003
version            NaN
bright_t31   -0.053730
frp          -0.035229
daynight      0.007317
fire_type     1.000000
Name: fire_type, dtype: float64


In [18]:
av1 = column_renamer(av1, ['bright_ti4', 'bright_ti5'], ['brightness', 'bright_t31'])
nrt_v1 = column_renamer(nrt_v1, ['bright_ti4', 'bright_ti5'], ['brightness', 'bright_t31'])

In [19]:
am6.drop(columns='daynight', inplace=True)
nrt_m6.drop(columns='daynight', inplace=True)

In [20]:
df = pd.merge(av1, am6, how='outer')

In [21]:
X, y = targeter(df, 'fire_type')

In [22]:
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=151)

In [47]:
seed = 4600

# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('PR', LinearRegression()))
models.append(('RFR', RandomForestClassifier()))
models.append(('XGB', XGBClassifier()))

# evaluate each model in turn
results = []
names = []
scoring = ['mean_square_error']
for name, model in models:
    
    kfold = KFold(n_splits=4, shuffle=True, random_state=seed)

    """if name == 'PR':  # Por utilizar el modelo del polinomio
        poly_reg = PolynomialFeatures(degree = 4)
        X_poly = poly_reg.fit_transform(X_train)
        cv_results = cross_val_score(model, X_poly, y_train.ravel(), cv=kfold, scoring=scoring)
        pickle.dump(model, open(f'GRID-{name}.sav', 'wb'))
    else:"""  # Para cualquier otro caso
    cv_results = cross_val_score(model, X_train, y_train.ravel(), cv=kfold, scoring=scoring)
    pickle.dump(model, open(f'GRID3-{name}.sav', 'wb'))
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison 
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

ValueError: For evaluating multiple scores, use sklearn.model_selection.cross_validate instead. ['mean_square_error'] was passed.

In [59]:
# Create a pipeline

pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])


logistic_params = {
    'classifier': [LogisticRegression(warm_start=True)],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.logspace(0, 4, 10)
    }

random_forest_params = {
    'classifier': [RandomForestClassifier(warm_start=True)],
    'classifier__n_estimators': [10, 100, 1000],
    'classifier__max_features': [1, 2, 3]
    }
    
xgboost_params = {
    'classifier' : [XGBClassifier(warm_start=True)],
    'classifier__learning_rate'  : [0.10, 0.15, 0.2],
    'classifier__max_depth'        : [4, 5, 7],
    'classifier__min_child_weight' : [ 3, 5, 7],
    'classifier__gamma'            : [ 0.1, 0.2, 0.3],
    'classifier__colsample_bytree' : [ 0.4, 0.5, 0.7]}

# hypertuning 
# Create space of candidate learning algorithms and their hyperparameters
search_space = [
    logistic_params,
    random_forest_params,
    xgboost_params
    ]

In [60]:
%%time

cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
# Create grid search 
clf = GridSearchCV(estimator=pipe, param_grid=search_space, cv=cv, verbose=0, n_jobs=-2)

# Fit grid search
best_model = clf.fit(X_train, y_train)

# View best model
separator = "\n############################\n"
print(separator)
print("best estimator:", best_model.best_estimator_.get_params()['classifier'])
print(separator)
print("clf.best_params_", clf.best_params_)
print(separator)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_)
#SAVE MODEL
# save the model to disk
filename = 'hyperXGB6.sav'
success = best_model.best_estimator_.get_params()
pickle.dump(success, open(filename, 'wb'))